In [5]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-130254
Azure region: southcentralus
Subscription id: a0f586ec-8016-4ea9-8248-9bf2299ad437
Resource group: aml-quickstarts-130254


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
from azureml.core.compute_target import ComputeTargetException

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
vm_size = "Standard_D2_V2"


# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
        "--C": uniform(0.1,1),
        "--max_iter": choice(50, 100, 150, 200)
    })### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1, delay_evaluation = 5)### YOUR CODE HERE ###

#if "training" not in os.listdir():
#    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.', compute_target=cpu_cluster, 
              entry_script='train-Copy1.py')### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy= policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=100,
                                max_concurrent_runs=4)
                                ### YOUR CODE HERE###

In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
###HyperDriveRun(experiment, run_id, hyperdrive_config=hyperdrive_config)
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_5c7b4a8a-5681-4508-9dfe-ba96ee59a047
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_5c7b4a8a-5681-4508-9dfe-ba96ee59a047?wsid=/subscriptions/a0f586ec-8016-4ea9-8248-9bf2299ad437/resourcegroups/aml-quickstarts-130254/workspaces/quick-starts-ws-130254

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-12T21:10:38.802395][API][INFO]Experiment created<END>\n""<START>[2020-12-12T21:10:39.387992][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2020-12-12T21:10:39.7359087Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-12-12T21:10:39.829932][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [6]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best run ID: ', best_run.id)
print('\n Metrics: ', best_run_metrics)
print('\n Parameters: ', parameter_values)

Best run ID:  HD_034a3669-3e62-4262-bbb0-0cbaf8cad518_19

 Metrics:  {'Regularization Strength:': 0.21353303026327866, 'Max iterations:': 50, 'Accuracy': 0.9111642449880448}

 Parameters:  ['--C', '0.21353303026327866', '--max_iter', '50']


In [7]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_034a3669-3e62-4262-bbb0-0cbaf8cad518_19,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [8]:
best_run.id

'HD_034a3669-3e62-4262-bbb0-0cbaf8cad518_19'

In [10]:
os.makedirs('outputs', exist_ok = True)
joblib.dump(value = best_run.id, filename = 'outputs/model.joblib')

['outputs/model.joblib']

In [11]:
# check 
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_08f933f26c850e9637792f6d442ff5f56ee959cba438bd4257fb8280541a68f8_p.txt',
 'azureml-logs/65_job_prep-tvmps_08f933f26c850e9637792f6d442ff5f56ee959cba438bd4257fb8280541a68f8_p.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_08f933f26c850e9637792f6d442ff5f56ee959cba438bd4257fb8280541a68f8_p.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/106_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log']

In [13]:
# register the best model 
#best_run.register_model(model_name = 'best_regression_model', model_path = 'outputs/model.joblib')

In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
from azureml.core.dataset import Dataset
url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
dataset2 = TabularDatasetFactory.from_delimited_files(path = url)


In [15]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset2) ### your code###

In [16]:
### your code here###
from sklearn.model_selection import train_test_split

import pandas as pd

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.33)

df_train = pd.concat([x_train, y_train], axis=1)

In [17]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_settings = {
    #'enabling_early_stopping': True,
    'iteration_timeout_minutes': 5,
    'max_concurrent_iterations': 4,
    'featurization': 'auto'    
}

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=df_train,
    label_column_name='y',
    n_cross_validations=4,
    enable_voting_ensemble = False,
    enable_stack_ensemble = False, 
    **automl_settings)

In [18]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

exp2 = Experiment(workspace=ws, name="automl-udacity-project")
remote_run = exp2.submit(automl_config, show_output=True)

RunDetails(remote_run).show()
remote_run.get_status()
remote_run.wait_for_completion(show_output=True)

Running on local machine
Parent Run ID: AutoML_44f8cc49-93eb-4c11-b178-372c29a02717

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|2473                             |1                                |22076                                 |
+---------------------------------+----------

{'runId': 'AutoML_44f8cc49-93eb-4c11-b178-372c29a02717',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-11-16T03:56:13.361121Z',
 'endTimeUtc': '2020-11-16T04:26:20.16333Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-udacity-project","subscription_id":"de47103e-2da6-4f5e-88fc-d18b27fd249b","resource_group":"aml-quickstarts-126413","workspace_name":"quick-starts-ws-126413","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":4,"y_min":n

In [20]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)
print(fitted_model._final_estimator)

Run(Experiment: automl-udacity-project,
Id: AutoML_44f8cc49-93eb-4c11-b178-372c29a02717_0,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('MaxAbsScaler', MaxAbsScaler(copy...
                 LightGBMClassifier(boosting_type='gbdt', class_weight=None,
                                    colsample_bytree=1.0,
                                    importance_type='split', learning_rate=0.1,
                                    max_depth=-1, min_

In [21]:
joblib.dump(value = fitted_model, filename = 'outputs/automl-model.joblib')

['outputs/automl-model.joblib']

In [23]:
cpu_cluster.delete()

ComputeTargetException: ComputeTargetException:
	Message: Received bad response from Resource Provider:
Response Code: 400
Headers: {'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Length': '265', 'Content-Type': 'application/json; charset=utf-8', 'Expires': '-1', 'x-ms-correlation-request-id': 'c8229dec-bec3-4a39-b467-b66f83f32e34', 'x-ms-ratelimit-remaining-subscription-deletes': '14998', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-response-type': 'standard', 'x-ms-request-id': '|00-4a2e8fdf5ed6c54fa57096f07cdf9b29-028d974193fd5449-00.e81d761a_', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.451', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'x-ms-routing-request-id': 'SOUTHCENTRALUS:20201116T052250Z:c8229dec-bec3-4a39-b467-b66f83f32e34', 'Date': 'Mon, 16 Nov 2020 05:22:49 GMT'}
Content: b'{"error":{"code":"ResourceNotReady","message":"Compute with name cpu-cluster is already being deleted.","innererror"